In [7]:
# First let's create our PySpark instance
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("FD_ML").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
appid = spark._jsc.sc().applicationId()
print("You are working with", cores, "core(s) on appid: ",appid)
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You are working with 3 core(s) on appid:  application_1742332084551_0004

In [8]:
# Install pandas and numpy in EMR cluster
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1742332084551_0004/container_1742332084551_0004_01_000001/pyspark.zip/pyspark/context.py", line 2614, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [9]:
path = "s3://fd-final/fraud_data/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
path = "s3a://fd-final/fraud_data/fraud_processed.parquet"

fraud_data_df = spark.read.parquet(path)

fraud_data_df.printSchema()
fraud_data_df.show(5, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: integer (nullable = true)
 |-- scaled_features: vector (nullable = true)

+-----+------------------------------------------------------------------------------------------------------------------+
|label|scaled_features                                                                                                   |
+-----+------------------------------------------------------------------------------------------------------------------+
|0    |[0.0013268461796048614,0.75,0.47058823529411764,0.9862637362637363,0.13043478260869565,0.6666666666666666,0.0,0.5]|
|0    |[0.011691847066007444,0.25,0.39215686274509803,0.7967032967032968,0.7391304347826086,0.3333333333333333,0.0,0.0]  |
|1    |[0.0507353201883113,0.75,0.43137254901960786,0.07967032967032968,0.8695652173913043,0.3333333333333333,1.0,0.5]   |
|0    |[0.004581716469360017,1.0,0.3529411764705882,0.1098901098901099,0.08695652173913043,1.0,1.0,1.0]                  |
|0    |[0.011393227884255518,1.0,0.6862745098039

In [11]:
# First - Read in dependencies
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.sql.functions import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#Classification

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Rename `scaled_features` to `features` so that the ML libraries model will be able to understand it as input column = features
fraud_data_df = fraud_data_df.withColumnRenamed("scaled_features", "features")

# Split into train and test splits
train, test = fraud_data_df.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Logistic Regression 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# load libraries 
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Set up evaluation objects
Bin_evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")  # AUC evaluation
MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy")  # Accuracy evaluation

# Logistic Regression Model
classifier = LogisticRegression(featuresCol="features", labelCol="label")

# Set up parameter grid for hyperparameter tuning
paramGrid = (ParamGridBuilder()
             .addGrid(classifier.maxIter, [10, 15, 20])
             .build())

# Set up Cross Validator
crossval = CrossValidator(estimator=classifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=MC_evaluator,
                          numFolds=3)  # 3+ is best practice

# Train the model
fitModel = crossval.fit(train)

# Collect the best model
BestModel = fitModel.bestModel
print("Intercept: " + str(BestModel.interceptVector))
print("Coefficients: \n" + str(BestModel.coefficientMatrix))

#Logistic Regression Identifer for ML Models 
LR_BestModel = BestModel

# Make predictions on the test set
predictions = fitModel.transform(test)

# Compute accuracy
accuracy = MC_evaluator.evaluate(predictions) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Compute AUC for binary classification
auc = Bin_evaluator.evaluate(predictions)
print(f"AUC LR: {auc:.4f}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-15:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


Intercept: [-1.970427840614607]
Coefficients: 
DenseMatrix([[ 3.01864629e+01, -7.06929575e-03, -2.98822835e-02,
              -2.26093627e+00, -1.77410818e+00,  1.16370881e-02,
              -2.72837998e-03, -1.17584872e-02]])
Accuracy: 95.47%
AUC LR: 0.5561

In [16]:
#Classification Report for Logistic Regression 

# Load the Summary
trainingSummary = LR_BestModel.summary

# General Describe
trainingSummary.predictions.describe().show()

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print(" ")
print("objectiveHistory: (scaled loss + regularization) at each iteration")
for objective in objectiveHistory:
    print(objective)

# for multiclass, we can inspect metrics on a per-label basis
print(" ")
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print(" ")
print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print(" ")
print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print(" ")
print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print(" ")
print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

# Generate confusion matrix and print (includes accuracy)
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print(" ")
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-16:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


+-------+-------------------+--------------------+
|summary|              label|          prediction|
+-------+-------------------+--------------------+
|  count|            1046711|             1046711|
|   mean|0.05016284342096338|0.006606408072524317|
| stddev|0.21828096133462427| 0.08101092342888183|
|    min|                0.0|                 0.0|
|    max|                1.0|                 1.0|
+-------+-------------------+--------------------+

 
objectiveHistory: (scaled loss + regularization) at each iteration
0.19899444701205124
0.17256204802835995
0.16720069933150394
0.1657257179246831
0.16508556860487592
0.16502899971421386
0.16502661147334755
0.1650265083167095
0.16502650486233728
0.16502650485356046
 
False positive rate by label:
label 0: 0.8850036186340609
label 1: 0.0008821118381018
 
True positive rate by label:
label 0: 0.9991178881618982
label 1: 0.11499638136593913
 
Precision by label:
label 0: 0.9553104647450077
label 1: 0.8731742588575561
 
Recall by label:


In [17]:
'''
According to the Classification Report for Logisitic Regression, accuracy was at 95.48%, precision by label("isFradulent) for class 0 was at 95.55%, class 1 was at
87.08%, recall by label, for class 0 at 99.91% and class 1 at 11.44%. Overall precision at 95.11% and recall at 95.48%. This is a high accuracy rate, 
I believe it may be better to increase the amount of folds in the future and preprocess the dataset more to look for any skewness and outliers. '''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\nAccording to the Classification Report for Logisitic Regression, accuracy was at 95.48%, precision by label("isFradulent) for class 0 was at 95.55%, class 1 was at\n87.08%, recall by label, for class 0 at 99.91% and class 1 at 11.44%. Overall precision at 95.11% and recall at 95.48%. This is a high accuracy rate, \nI believe it may be better to increase the amount of folds in the future and preprocess the dataset more to look for any skewness and outliers. '

In [18]:
#Decision Tree 

VBox()

Exception in thread cell_monitor-17:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
classifier = DecisionTreeClassifier()
paramGrid = (ParamGridBuilder()                               
             .addGrid(classifier.maxDepth, [5, 10, 20])  
             .addGrid(classifier.maxBins, [10, 40])      
             .build())

#Cross Validator requires all of the following parameters:
crossval = CrossValidator(estimator=classifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3) # 3 + is best practice
# Fit Model: Run cross-validation, and choose the best set of parameters.
fitModel = crossval.fit(train)

# Collect and print feature importances
BestModel = fitModel.bestModel
featureImportances = BestModel.featureImportances.toArray()
print("Feature Importances: ",featureImportances)

#Decision Tree Identifer for ML Models 
DT_BestModel = BestModel

predictions = fitModel.transform(test)

# Compute accuracy
accuracy = MC_evaluator.evaluate(predictions) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Compute AUC for binary classification
auc = Bin_evaluator.evaluate(predictions)
print(f"AUC DT: {auc:.4f}")

VBox()

Exception in thread cell_monitor-18:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-19:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


Feature Importances:  [4.60421360e-01 0.00000000e+00 6.51650084e-05 4.11730192e-01
 1.27783283e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Accuracy: 95.39%
AUC DT: 0.5623

In [20]:
#Classification Report for Decision Tree

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize evaluators
evaluator_accuracy = MulticlassClassificationEvaluator(metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(metricName="weightedRecall")

# Compute evaluation metrics for Decision Tree
accuracy = evaluator_accuracy.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)

# Print results
print("Decision Tree Model Evaluation")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-20:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


Decision Tree Model Evaluation
Accuracy: 0.9539
F1 Score: 0.9382
Precision: 0.9448
Recall: 0.9539

In [21]:
'''
According to our CLassification Report for Decision Tree, overall accuracy was at 95.43%, precision was at 94.67%, recall was at 95.43%. This result 
shows a high accuracy which can be reduced if we did more preprocessing on the dataset, additionally we have have a lower accuracy if change our parameters 
or cross val folds since this is a large dataset.'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\nAccording to our CLassification Report for Decision Tree, overall accuracy was at 95.43%, precision was at 94.67%, recall was at 95.43%. This result \nshows a high accuracy which can be reduced if we did more preprocessing on the dataset, additionally we have have a lower accuracy if change our parameters \nor cross val folds since this is a large dataset.'

In [22]:
#Naive-Bayes 

VBox()

Exception in thread cell_monitor-21:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Add parameters of your choice here:
classifier = NaiveBayes()
paramGrid = (ParamGridBuilder() \
             .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6]) \
             .build())

#Cross Validator requires all of the following parameters:
crossval = CrossValidator(estimator=classifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=2) # 3 + is best practice
# Fit Model: Run cross-validation, and choose the best set of parameters.
fitModel = crossval.fit(train)

#Naives-Bayes Identifer for ML Models 
NB_BestModel = BestModel

predictions = fitModel.transform(test)

# Compute accuracy
accuracy = MC_evaluator.evaluate(predictions) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Compute AUC for binary classification
auc = Bin_evaluator.evaluate(predictions)
print(f"AUC DT: {auc:.4f}") #AUC NB 

Exception in thread cell_monitor-22:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-23:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


Accuracy: 94.99%
AUC DT: 0.5000

In [24]:
#Classification Report for Naive-Bayes

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize evaluators
evaluator_accuracy = MulticlassClassificationEvaluator(metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(metricName="weightedRecall")

# Compute evaluation metrics for Naive-Bayes
accuracy = evaluator_accuracy.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)

# Print results
print("Naive-Bayes Evaluation")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-24:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


Naive-Bayes Evaluation
Accuracy: 0.9499
F1 Score: 0.9254
Precision: 0.9022
Recall: 0.9499

In [ ]:
'''
According to our CLassification Report for Naive-Bayes, overall accuracy was at 94.99%, precision was at 90.22%, recall was at 94.99%. This result 
shows a high accuracy which can be reduced if we did more preprocessing on the dataset such as more binning and smoothing techniques. 
'''

In [ ]:
'''Therefore, all models performed similarly with logistic regression with accuracy at 95.47%, decision tree with accuracy at 95.39%, and Naive-Bayes 
with accuracy at 94.99%.'''